In [ ]:
import os
import datetime

import pandas as pd
import numpy as np
from scipy.signal import savgol_filter

from duneAnalysis import duneAnalysis

import matplotlib.pyplot as plt

## Import input data and define variables

For this example test data is given in the folder 'Example_data'.

All output will be stored as pandas dataframes in pickle files.

In [ ]:
# define the data folder (MiddenWaal, or TielstAndries, or LowerWaal)
dataFolder = r'Data'

# define the output folder for the wavelet output
outFolder = os.path.join(dataFolder,f'duneParam')
if not os.path.exists(outFolder):
    os.mkdir(outFolder)

# read the data from example_data, convert strings in DatesFrame to datetime values
ZFrame = pd.read_csv(os.path.join(dataFolder,'Z_locations.csv')
                     ,index_col=0,skiprows = 6)
MXYFrame = pd.read_csv(os.path.join(dataFolder,'MXY_locations.csv')
                       ,index_col=0,skiprows = 6)
DatesFrame = pd.read_csv(os.path.join(dataFolder,'dates.csv')
                         ,index_col=0,skiprows = 6)
for i,_ in enumerate(DatesFrame.columns):
    DatesFrame.iloc[:,i] = pd.to_datetime(DatesFrame.iloc[:,i])

# make the values of M and Z a numpy arrays
Z = ZFrame.values
M = MXYFrame.M.values

# define the maximum and minimum wavelengths that may be considered dunes, for the filtering of the wavelet
# lower limit: to exclude ripples, upper limit based on the expected dunes 
# eg. Wilbers and Ten brinke, 2003; Frings and Kleinhans, 2008; de Ruijsscher et al., 2020; Zomer et al., 2021
upperDuneLimit = 300
lowerDuneLimit = 20

# Define the shifts and the files variables for the spatial cross=correlation,
# Shifts are used to determine the dune celerity for a 1000 m long section
# files are in this example the column names of Zframe and DatesFrame, these columns linkt those varables
# to the date of measuring of each point.
frameSize  = 1000 #Perform the analysis over sections of 1000 m long
frameShift = 1000
shifts = np.arange((np.shape(M)[0]+1-frameSize)//frameShift)

files = ZFrame.columns.values

In [ ]:
# create the filtered dune profiles, without the 
Zfilt = savgol_filter(Z,41,3,mode='mirror',axis = 0)

# Dune Analysis; Dune shape using the wavelet method

## Requirements and principles wavelet method
- Wavelet analysis performed over the whole signal, unfitlered, only NaNs should be replaced by values. Either by interploation (in the midddle of the signal) or by extrapolation/duplication of last not NaN value (at the edges of the signal)
- The wavelet spectrum (class wavelet) can be saved if this is needed for further analysis
- Reconstruct the dunes based on the length scales found in the wavelet spectrum, for the Waal system this the range of scales to create the reconstruction is between 20 m and 300 m. Ripples in the Waal have a length of 5 to 10 m, so the smallest scale of 20 meter ensures ripples will not be recontstructed. Based on a study of the wavelet spectra for different flow regimes the maximum scale should be 300 m. The scales are determined based on the wavelet spectra and a short study of the dune profiles. The scales can be changed to different values for different systems.
- Determine the crest locations in the reconstructed profiles.
- When studying dunes filter the original signal with a Savitski-Golay filter with lengththscale about 4 times the expected lengthscales of teh ripples in the bed profile. And polynomial 3. Such that the ripples are filtered out and the location of the crest of the primary dune is not altered too much.
- To determine the crest locations, correlate the crest locations found in the wavelet-reconstructed profile with the crest locations (local maxima) in the filtered-original profile. The location the local maximum in the origninal-filterd profile closest to each peak in the wavelet filtered signal is then the final crest location.
- Throughs are points with the lowest eleveation between two crests.


## Input

- matrix/numpy 2d-array with bed elevation (Z), at each row bed elevation at a location, each column is one measurement (time), or one line in the meausurement
- Filtered profile (Zfilt)
- Location of the measurements along one line (M), dM must be constant along the line.

## Output
Crest and Trough locations are given as index values for each column in Z/Zfilt.

In [ ]:
Zreconstructed = np.zeros(np.shape(Z))
dnums = np.arange(np.shape(Z)[1])

CrestTroughs = pd.DataFrame(columns = ['Crests','Troughs'], index = dnums)
  
for d in dnums:
    Mcrests,Mtroughs,Zreconstructed[:,d],_ = duneAnalysis.Wavelet2Dunes(Z[:,d],
                                                             Zfilt[:,d],
                                                             M = M[:], 
                                                             lowLim = lowerDuneLimit, 
                                                             upLim = upperDuneLimit)
    CrestTroughs.loc[d,'Crests']   = [Mcrests]
    CrestTroughs.loc[d,'Troughs']  = [Mtroughs]

# # save the locations of the crests and trhoughs in a pickle
# CrestTroughs.to_pickle(os.path.join(outFolder,'CrestTroughLocations_2011-2021_centerline.pkl'))

## Determine the dune shape parameters
The dune shape parameters can be determined for each identified dune (DuneParam) and as statistics over all dunes in one profile, column in Z, in DuneParamStat. The following dune parameters are determined

- Dune length, L, horizontal distance between two consequtive troughs in m.
- Dune height, H, vertical distancs between a crest and its downstream trough in m.
- Aspect ratio, LH, dune height/dune length. Determined for each individual dune and then statistics are determined in m/m.
- Lee slope angle, Slee, the mean angle of the middle 4/6th section of the lee slope, in degrees.

## Input
- Zfilt,
- M,
- crest locations,
- trough locations

## Output
- dataframe DuneParam; dune parameters for each individual dune in each profile
- dataframe DuneParamStat; statistics of dune parameters in each profile. Mean, 5%, 25%, 75% and 95% bounds of the data


In [ ]:
# create a dataframe for the dune shape parameters
DuneParam = pd.DataFrame(columns = ['Lengths','Heights','LHratios','Slee'], index = dnums)
DuneParamStat = pd.DataFrame(columns = ['L','L2','L05','L25','L75','L95',
                                        'H','H05','H25','H75','H95',
                                        'LH','LH05','LH25','LH75','LH95',
                                        'Slee','Slee05','Slee25','Slee75','Slee95'],index = dnums)
CrestTroughs.Crests.loc[0][0]

frameSize  = np.shape(M)[0] # if one wants to analyse different parts of the data
frameShift = 1000

for d in dnums:
    crests = CrestTroughs.Crests.loc[(d)][0]
    troughs = CrestTroughs.Troughs.loc[(d)][0]
    # at least 1 complete dune must be present in the signal
    if len(crests) < 2:
        continue
    if len(troughs) < 2:
        continue

    # determine the dune parameters
    lengths,heights,lhratio,Slee = duneAnalysis.duneHeightLengthRatio(Zfilt[:,d], M, crests, troughs)

    # store the parameters in the predefined dataframes
    DuneParam.loc[(d),'Lengths']  = lengths
    DuneParam.loc[(d),'Heights']  = heights
    DuneParam.loc[(d),'LHratios'] = lhratio
    DuneParam.loc[(d),'Slee']     = Slee

    DuneParamStat.loc[(d),'L'] = np.mean(lengths)
    DuneParamStat.loc[(d),'H'] = np.mean(heights)
    DuneParamStat.loc[(d),'LH'] = np.mean(lhratio)
    DuneParamStat.loc[(d),'Slee'] = np.mean(Slee)

    DuneParamStat.loc[(d),['L05','L25','L75','L95']] = np.percentile(lengths,[5,25,75,95])
    DuneParamStat.loc[(d),['H05','H25','H75','H95']] = np.percentile(heights,[5,25,75,95])
    DuneParamStat.loc[(d),['LH05','LH25','LH75','LH95']] = np.percentile(lhratio,[5,25,75,95])
    DuneParamStat.loc[(d),['Slee05','Slee25','Slee75','Slee95']] = np.percentile(Slee,[5,25,75,95])
# save the parameters in a pickle
DuneParam.to_pickle(os.path.join(outFolder,f'DuneParam_all_2011-2021_centerline.pkl')) #{istart:05d}_{iend:05d}
DuneParamStat.to_pickle(os.path.join(outFolder,f'DuneParamStat_all_2011-2021_centerline.pkl'))

# Dune Analysis; celerity using spatial cross-correlation
The spatial cross correlation function (lag_finder), determines the the displacement of the profile based on the cross correlation values of one profile moving over the previous profile.

- Remove large scale morhpology (point bars, larger bar structures, erosion holes/sedimentation pits linked to river geometry at a larger scale than the dunes)
- Make dure the time dune haven't changed too much in shape or have moved more than one dune length between two consequtive profiles; This may be the case if the time interval is too large or during floodwaves when the dunes move fast

To determine dune celerity, the displacement is divided over the numer of days between the two measurements.

## Input
- Zfilt
- Z
- shifts; number of shifts, frameShift: step for each frame. frameSize: length of each frame
- DatesFrame; date of measurement for each value in Z.

## Output
4 dataframes:
- duneSpeeds: dune celerity in each frame per measurement
- duneDisplacement: displacement of the profile in each frame per measurement 
- duneSpeedCorr: correlation factor found for the displacements
- duneSpeedShiftDates: dates of measurements for each frame per measurement, determinde as the date that occured most often in that frame.

In [ ]:
# create the signal without the underlying bed 1001 may be chosen differently if the underlying bed is more variable
Zdunes = Zfilt - savgol_filter(Z,1001,3,mode='mirror',axis = 0)

# define dataframes to put data in
duneSpeeds       = pd.DataFrame(columns = files, index = shifts) # displacement/dt
duneDisplacement = pd.DataFrame(columns = files, index = shifts) # calculated displacements
duneSpeedCorr    = pd.DataFrame(columns = files, index = shifts) # correlation at the displacement
duneSpeedShiftDates = pd.DataFrame(columns = files, index = shifts)  # dt

noval = datetime.datetime.strptime('2000-01-01', '%Y-%m-%d')

for shift in shifts:
    shiftStart = int(shift*frameShift)
    shiftEnd = int(shift*frameShift+frameSize)
    shiftdates = DatesFrame.iloc[shiftStart:shiftEnd].mode().values[0]
    
    duneSpeedShiftDates.iloc[shift] = shiftdates
          
    for f,file in enumerate(files[:-1]):
        # dune speed by determining the spatial lag based in spatial cross-correlation
        if (shiftdates[f]!= noval) & (shiftdates[f+1]!= noval):
            # normal routine, assign displacement, correlation to date f+1
            displ,corr = duneAnalysis.lag_finder(Zdunes[shiftStart:shiftEnd,f],
                                                 Zdunes[shiftStart:shiftEnd,f+1],
                                                 1, thresholdMin = 5) 
            duneDisplacement.loc[(shift),files[f+1]] = displ
            duneSpeedCorr.loc[(shift),files[f+1]] = corr
            
            dt = shiftdates[f+1]-shiftdates[f]
            dt = dt.astype('timedelta64[D]').astype(int)

            duneSpeeds.loc[(shift),files[f+1]] = duneDisplacement.loc[(shift),files[f+1]]/dt
        elif (shiftdates[f] == noval) & (shiftdates[f+1] != noval):
            displ,corr = duneAnalysis.lag_finder(Zdunes[shiftStart:shiftEnd,f-1],
                                                 Zdunes[shiftStart:shiftEnd,f+1],
                                                 1, thresholdMin = 5)
            
            duneDisplacement.loc[(shift),files[f+1]] = displ
            duneSpeedCorr.loc[(shift),files[f+1]] = corr
            
            duneDisplacement.loc[(shift),files[f]] = np.nan
            duneSpeedCorr.loc[(shift),files[f]] = np.nan

            dt = shiftdates[f+1]-shiftdates[f-1]
            dt = dt.astype('timedelta64[D]').astype(int)
            duneSpeeds.loc[(shift),files[f+1]] = duneDisplacement.loc[(shift),files[f+1]]/dt
            duneSpeeds.loc[(shift),files[f]]   = np.nan
        elif (shiftdates[f] != noval) &(shiftdates[f+1] == noval):
            # this issue will be solved in case above
            continue
        elif (shiftdates[f]== noval) & (shiftdates[f+1]== noval):
            duneDisplacement.loc[(shift),files[f]] = np.nan
            duneSpeedCorr.loc[(shift),files[f]] = np.nan
            duneSpeeds.loc[(shift),files[f]]   = np.nan

            duneDisplacement.loc[(shift),files[f+1]] = np.nan
            duneSpeedCorr.loc[(shift),files[f+1]] = np.nan
            duneSpeeds.loc[(shift),files[f+1]]   = np.nan

# replace the 0 values in the column without 
duneSpeedShiftDates = duneSpeedShiftDates.replace(to_replace=0, method='bfill')

duneSpeeds.to_pickle(os.path.join(outFolder,f'DuneSpeeds_all_2011-2021_centerline.pkl'))
duneSpeedCorr.to_pickle(os.path.join(outFolder,f'duneSpeedCorr_all_2011-2021_centerline.pkl'))
duneDisplacement.to_pickle(os.path.join(outFolder,f'duneDisplacement_all_2011-2021_centerline.pkl'))
duneSpeedShiftDates.to_pickle(os.path.join(outFolder,f'duneSpeedShiftDates_all_2011-2021_centerline.pkl'))
 